In [12]:
import tensorflow as tf
import numpy as np
from keras import backend as K
from keras.utils import plot_model, layer_utils
from keras import layers
from keras.layers import (Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, concatenate, MaxPooling2D, AveragePooling2D)
from keras.models import Model, load_model
from keras_preprocessing import image
import matplotlib.pyplot as plt


print("Loading data")
data = tf.keras.datasets.fashion_mnist.load_data()
training_data, testing_data = data
x_train, y_train = training_data
x_test, y_test = testing_data
print(x_test.shape)
print(x_train.shape)

def stacked_block(X, filters, index, block):
    """
    Arguments:
    X : index
    filters: encodes the no. of filters in each layer
    [1x1,1x1(3),3x3,1x1(5),5x5,1x1(7),7x7,1X1(9),9X9]
    index, block: for naming purposes
    """
    base_name = "stk_blk"+str(index)+block
    X1 = Conv2D(filters[0], [1, 1], padding="same", name=base_name+"1x1")(X)
    X2 = Conv2D(filters[1], [1, 1], padding="same", name=base_name+"1x1-3")(X)
    X2 = Conv2D(filters[2], [3, 3], padding="same", name=base_name+"3x3")(X2)
    X3 = Conv2D(filters[3], [1, 1], padding="same", name=base_name+"1x1-5")(X)
    X3 = Conv2D(filters[4], [5, 5], padding="same", name=base_name+"5x5")(X3)
    X = concatenate([X1, X2, X3], axis=3)

    return X


def identity_block(X, block, modifier=None):
    if modifier is None:
        modifier = 1
    X_init = X
    filter_size = np.array([128, 64, 128, 64, 32,])
    filter_size = modifier*filter_size
    aux_size = np.sum(filter_size[::2])
    print(aux_size)
    print(filter_size)
    X = stacked_block(
        X, filter_size, block, "a")
    X = Activation("relu")(X)
    X = stacked_block(
        X, filter_size, block, "b")
    X = Activation("relu")(X)
    X = stacked_block(
        X, filter_size, block, "c")
    X_init = Conv2D(aux_size, [
                   1, 1], name="aux_conv"+str(block))(X_init)
    X = Add()([X_init, X])
    X = BatchNormalization(axis=3, name=block+"bnorm")(X)
    X = Activation("relu")(X)
    return X


def gen_model(input_shape):
    X_input = Input(input_shape)
    X = Conv2D(256, (3, 3), padding="same", name="conv0")(X_input)
    X = Activation("relu")(X)
    X = identity_block(X, "A")
    X = MaxPooling2D((2, 2),strides=(2,2), name="max_pool1")(X)
    X = identity_block(X, "B")
    X = MaxPooling2D((3, 3),strides=(2,2), name="max_pool2")(X)
    X = Flatten()(X)
    X = Dense(256, activation="relu", name="fc1")(X)
    X = Dense(128, activation="sigmoid", name="fc2")(X)
    X = Dense(10, activation="softmax", name="output")(X)
    model = Model(inputs=X_input, outputs=X, name="MNIST_clothes")
    return model


print("Initialized")
model = gen_model((28, 28, 1))
model.compile(optimizer="adam",loss="logcosh",metrics=["accuracy"])

Loading data
(10000, 28, 28)
(60000, 28, 28)
Initialized
288
[128  64 128  64  32]
288
[128  64 128  64  32]


In [14]:
model.summary()
model.load_weights("weights.h5")
plot_model(model, to_file="model.png")

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 28, 28, 256)  2560        input_5[0][0]                    
__________________________________________________________________________________________________
activation_29 (Activation)      (None, 28, 28, 256)  0           conv0[0][0]                      
__________________________________________________________________________________________________
stk_blkAa1x1-3 (Conv2D)         (None, 28, 28, 64)   16448       activation_29[0][0]              
__________________________________________________________________________________________________
stk_blkAa1

In [9]:
from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
lb.fit([0,1,2,3,4,5,6,7,8,9])
y_train_temp = lb.transform(y_train)
print(y_train_temp.shape)
x_train_temp =np.expand_dims(x_train,axis = 3)
y_test_temp = lb.transform(y_test)
x_test_temp = np.expand_dims(x_test,axis = 3)
for i in range(20):
  print(model.evaluate(x=x_test_temp[-2000:], y=y_test_temp[-2000:],verbose=1))
  model.fit(x=np.flip(x_train_temp,axis=2),y=y_train_temp,epochs = 2, batch_size=512)
  model.fit(x=x_train_temp, y=y_train_temp, epochs = 2, batch_size = 256)
  model.save("weights.h5")
  print("iteration :{} finished".format(i))

(60000, 10)
2000/2000 [==============================] - 3s 1ms/step
[0.004386550415714737, 0.939]
Epoch 1/2
  512/60000 [..............................] - ETA: 3:21 - loss: 0.0040 - acc: 0.9473

KeyboardInterrupt: ignored